In [ ]:
import copy
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import h5py
import uuid
from datetime import datetime
from qutip import *
import sparse as sp

mpl.rcParams['figure.dpi'] = 200

# First Seed
This notebook demonstrates the initial seeding process where we start in all $|g>$ and a THz-photon pushes one atom in $|r>$. Afterwards this atom should generate somewhere else a $|e>$ excitation through dipole-dipole interaction (between er).

This state is later on the starting point for the avalanche (other notebook).

## Define Constants

In [ ]:
N_spins = 8
N_phonons = 3

OmegaGE = 1
kappa = 0
omegaTrap = 8
V_vdw = 500
DeltaEE = -V_vdw  # weil resonant

tlist = np.linspace(0, 10, 101)

In [ ]:
def multiSpinOperator(op, numOfParticles, dim=2):
    retOp = []
    for i in range(numOfParticles):
        if i == 0:
            retOp.append(op)
        else:
            retOp.append(qeye(dim))
        for j in range(numOfParticles - 1):
            if j + 1 == i:
                retOp[i] = tensor(retOp[i], op)
            else:
                retOp[i] = tensor(retOp[i], qeye(dim))
    return retOp

In [ ]:
g = basis(2, 0)
e = basis(2, 1)
a = destroy(N_phonons)
g

## Create Hamiltonian

In [ ]:
#tempOpReshape = qeye([2, N_phonons])
tempOpReshape = qeye(2 * N_phonons)
tempOpReshape.data = tensor(e * g.dag() + g * e.dag(), qeye(N_phonons)).data
H_OmegaGE = OmegaGE * sum(multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0]))
tempOpReshape.data = tensor(e * e.dag(), qeye(N_phonons)).data
H_DeltaEE = DeltaEE * sum(multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0]))
tempOpReshape.data = tensor(qeye(2), num(N_phonons) + 0.5).data
H_phon = omegaTrap * sum(multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0]))

In [ ]:
H_OmegaGE

In [ ]:
tempOpReshape.data = tensor(e * e.dag(), qeye(N_phonons)).data
multiEe = multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0])

tempOpReshape.data = tensor(e * e.dag(), a).data
multiNa = multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0])

tempOpReshape.data = tensor(e * e.dag(), a.dag()).data
multiNaDag = multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0])

In [ ]:
H_vdw = 0 * H_DeltaEE
for i in range(N_spins):
    H_vdw += V_vdw * multiEe[i] * multiEe[(i + 1) % N_spins]
    #H_vdw -= V_vdw/ np.sqrt(omegaTrap) * (multiNaDag[i] * multiEe[(i + 1) % N_spins]
    #                                       + multiNa[i] * multiEe[(i + 1) % N_spins]
    #                                       - multiNa[(i + 1) % N_spins] * multiEe[i]
    #                                       - multiNaDag[(i + 1) % N_spins] * multiEe[i])
    H_vdw -= kappa * (multiNaDag[i] * multiEe[(i + 1) % N_spins]
                      + multiNa[i] * multiEe[(i + 1) % N_spins]
                      - multiNa[(i + 1) % N_spins] * multiEe[i]
                      - multiNaDag[(i + 1) % N_spins] * multiEe[i])

In [ ]:
H = H_OmegaGE + H_DeltaEE + H_vdw + H_phon
H

## Initial state
All atoms in $|g>$ or one in $|r>$

In [ ]:
g0 = basis(2 * N_phonons, 0)
e0 = basis(2 * N_phonons, N_phonons)
psiSpin = g0
for i in range(N_spins - 1):
    if i == N_spins // 2 - 1 or i == N_spins // 2 or i == N_spins // 2 - 2:
        psiSpin = tensor(psiSpin, e0)
    else:
        psiSpin = tensor(psiSpin, g0)
psi0 = psiSpin
e0

In [ ]:
# thus must not fail if entropy should be measured
entropy_mutual(psi0 * psi0.dag(), range(0, N_spins // 2), range(N_spins // 2, N_spins), sparse=False)

## expectation values

In [ ]:
eOps = []
tempOpReshape.data = tensor(e * e.dag(), qeye(N_phonons)).data
eOps += multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0])
tempOpReshape.data = tensor(qeye(2), num(N_phonons)).data
eOps += multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0])
tempOpReshape.data = tensor(qeye(2), destroy(N_phonons)).data
eOps += multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0])
tempOpReshape.data = tensor(qeye(2), create(N_phonons)).data
eOps += multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0])
# this is only to create an array with the same amount of observables (6) as in the mps-code
tempOpReshape.data = tensor(qzero(2), qzero(N_phonons)).data
eOps += multiSpinOperator(tempOpReshape, N_spins, dim=tempOpReshape.dims[0])
#eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 0), range(0, N_spins), sparse=False))
eOps.append(lambda t, psi: -1)
eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 1), range(1, N_spins), sparse=False))
eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 2), range(2, N_spins), sparse=False))
eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 3), range(3, N_spins), sparse=False))
eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 4), range(4, N_spins), sparse=False))
eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 5), range(5, N_spins), sparse=False))
eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 6), range(6, N_spins), sparse=False))
eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 7), range(7, N_spins), sparse=False))
eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 8), range(8, N_spins), sparse=False))
eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 9), range(9, N_spins), sparse=False))
eOps.append(lambda t, psi: entropy_mutual(psi * psi.dag(), range(0, 10), range(10, N_spins), sparse=False))

In [ ]:
res0 = sesolve(H, psi0, tlist, e_ops=eOps, progress_bar=True)  #,c_ops=c_ops)

In [ ]:
res0.expect

In [ ]:
res1 = copy.deepcopy(res0)
res1 = np.array(res1.expect).reshape(N_spins, 6, len(tlist), order='F')

## Plots

In [ ]:
plt.figure()
for i, r in enumerate(res1[:, 0, :]):
    plt.plot(tlist, r, label='<e' + str(i) + '>')
plt.title(r'Time evolution, $\Omega=$' + str(OmegaGE) + ', $\Delta=$' + str(DeltaEE) + ', $V=$' + str(V_vdw))
plt.xlabel('Time')
plt.ylabel('Expectation values')
plt.legend(loc='right')
plt.show()

In [ ]:
n_range = range(-N_spins // 2 + 1, N_spins // 2 + 1)
X, Y = np.meshgrid(tlist, n_range)
plt.pcolormesh(X, Y, res1[:, 0, :].real, cmap="viridis", vmin=0, vmax=1, shading='auto')
plt.title(r'exact: spins V={},$\Omega={},\omega={}$'.format(V_vdw, OmegaGE, omegaTrap))
plt.tight_layout()
plt.show()

In [ ]:
plt.pcolormesh(np.array(res1[:, 1, :].real), cmap="viridis", vmin=0, vmax=0.09)
plt.title(r'exact: phonons V={},$\Omega={},\omega={}$'.format(V_vdw, OmegaGE, omegaTrap))
plt.tight_layout()
plt.show()

In [ ]:
plt.figure()
#for i, r in enumerate(res1[:, 6, :]):
plt.plot(tlist, res1[N_spins // 2, 5, :], label='<e' + str(N_spins // 2) + '>')
plt.title(r'Time evolution, $\Omega=$' + str(OmegaGE) + ', $\Delta=$' + str(DeltaEE) + ', $V=$' + str(V_vdw))
plt.xlabel('Time')
plt.ylabel('Entanglement entropy')
plt.legend(loc='right')
plt.show()

In [ ]:
filename = 'comparisonWithMPS'
f = h5py.File(filename + '.h5', 'w')

# store now all calculated values to a hdf5 dataset
res_ds = f.create_dataset(uuid.uuid4().hex, data=res1, compression="gzip", compression_opts=9)
# TODO save observable data in separate dataframes
res_ds.attrs['observables'] = ['e', 'n_a', 'a', 'a_dag', 'bond_dim', 'entropy']
# store metadata which correspond to the dataset
res_ds.attrs['N_spins'] = N_spins
res_ds.attrs['N_phonons'] = N_phonons
res_ds.attrs['max_bond'] = -1
res_ds.attrs['Omega'] = OmegaGE
res_ds.attrs['omegaTrap'] = omegaTrap
res_ds.attrs['V_vdw'] = V_vdw
res_ds.attrs['DeltaEE'] = DeltaEE
res_ds.attrs['kappa'] = kappa
res_ds.attrs['CUTOFF_TOLERANCE'] = -1
res_ds.attrs['TROTTER_TOLERANCE'] = -1
res_ds.attrs['times'] = tlist
res_ds.attrs['Initial state'] = 'all atoms in g, centered in e, fock-state all 0'
res_ds.attrs['alpha'] = -1
res_ds.attrs['beta'] = -1
res_ds.attrs['Date'] = datetime.now().timestamp()
# close the file
f.close()